This notebook reproduces the **trajectory filtering, instantenous speed, and MSD analysis** described in the manuscript.  

## Analysis Workflow
- **Preprocessing**  
  - Walk through subfolders of TrackMate outputs (CSV files containing “edge” or “spot”).
        [NOTE:edge.csv is used to extract the instantaneous speed, and spot.csv is used to analyse MSD; assuming the csv files for each condition (stage*species) is concetenate into 1 file]
  - Clean headers and sort trajectories by time.
  - Add derived columns:
    - `time` (row index × frame interval, in minutes)
    - `index` (track identifier combining TRACK_ID and file name)
  - Plot SPEED vs. time for each trajectory and save figures.
  - Merge trajectories per subfolder and save as Excel files.


- **Filtering**  
  - Keep only trajectories with ≥ 31 time points (i.e., 150 min duration)
  - Truncate to first 31 time points for consistency  

- **MSD Computation**  
  - Project 2D droplet coordinates (X, Y) onto the principal motion axis using PCA  
  - Compute 1D mean squared displacement (MSD) across time lags  

- **Model Fitting**  
  - Fit MSD curves to a power-law model**:  
    \[
    MSD(\Delta t) \sim A \cdot (\Delta t)^\alpha
    \]  
  - Extract anomalous exponent α per droplet:  
    - α = 1 → Brownian diffusion  
    - α < 1 → Subdiffusion  
    - α > 1 → Superdiffusion  

- **Outputs**  
  - SPEED vs. time plots per merged csv
  - Merged and filtered Excel files of trajectories.
  - Per-droplet MSD curves (linear).
  - Per-droplet α values (Excel summary).
  - Population mean ± SD MSD curves with power-law fit.
  - Histogram of α distribution across droplets.

---


In [ ]:
# === Imports ===
import os
import itertools
from pathlib import Path
from statistics import mean

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

from scipy import stats
from scipy.optimize import curve_fit
from scipy.stats import gaussian_kde

from sklearn.decomposition import PCA
from matplotlib import cm
from matplotlib.colors import Normalize

import openpyxl

# Part1: instantenous speed analysis

### Step1 : Preprocessing and CSV merging

In [ ]:
base_folder = 'base_folder_name' ## structure: base_folder >> subfolder >> csv. files
output_folder_merged_data = "outpout_folder_name"

for root, dirs, files in os.walk(base_folder):
    edge_files = [f for f in files if f.endswith(".csv") and "edge" in f.lower()]
    
    if edge_files:
        print(f"\n📂 Processing subfolder: {root}")
        subfolder_name = os.path.basename(root.rstrip("/\\"))
        combined_dfs = []

        for file in edge_files:
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path, skiprows=[1, 2, 3]) # remove redundant headers
            df.columns = df.columns.str.strip()
            df = df.sort_values(by="EDGE_TIME").reset_index(drop=True)
            df["time"] = df.index * 5 # time interval: 5min
            base_name = os.path.splitext(file)[0]
            df["index"] = df["TRACK_ID"].astype(str) + "_" + base_name

            combined_dfs.append(df)

        if combined_dfs:
            pd.concat(combined_dfs, ignore_index=True).to_excel(
                f"{output_folder_merged_data}{subfolder_name}.xlsx", index=False
            )


### Step2 : Trajectory plotting

In [ ]:

figure_save = 'outout_figure_pathway'
file_path = "file_name.xlsx"
figure_name = os.path.splitext(os.path.basename(file_path))[0]


# --- Plot All Trajectories ---
df = pd.read_excel(file_path)
df['SPEED'] = df['SPEED']*60 # convert speed unit from micron/second into micron/minutes
grouped = df.groupby('index')
filtered_groups = {name: group.iloc[:31] for name, group in grouped if len(group) >= 31}
filtered_df = pd.concat(filtered_groups.values(), ignore_index=True)

plt.figure(figsize=(5, 4))
plt.rcParams['font.family'] = 'Arial'
for name, group in filtered_df.groupby('index'):
    plt.scatter(group['time'], group['SPEED'], s=20, alpha=0.8)
    plt.plot(group['time'], group['SPEED'], alpha=0.7, linewidth=0.7)

plt.xlabel('time', fontsize=10)
plt.ylabel('real-time-speed per displacement', fontsize=10)
plt.title(f'{figure_name} - All Trajectories', fontsize=12)
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(25))
plt.xlim(5, 150) 
plt.tight_layout()
plt.savefig(f"{figure_save}/{figure_name}.pdf")
plt.show()

summary_save_path = os.path.join(figure_save, f"{figure_name}_group_summary.txt")

# --- mean+std Trajectories ---

pivot_table = filtered_df.pivot_table(index='time', columns='index', values='SPEED')

mean_values = pivot_table.mean(axis=1)
std_values = pivot_table.std(axis=1)

plt.figure(figsize=(5, 4))
plt.plot(mean_values.index, mean_values.values, color='black', linewidth=2, label='Mean')
plt.fill_between(mean_values.index, mean_values - std_values, mean_values + std_values, color='gray', alpha=0.5, label='Mean ± SD')

plt.xlabel('time', fontsize=10)
plt.ylabel('real-time-speed per displacement', fontsize=10)
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(25))

plt.xlim(5, 150)
plt.ylim(-0.2, 1.6)

plt.title(f"{figure_name} - Normalized Mean ± SD", fontsize=12)
plt.legend()
plt.tight_layout()
plt.savefig(f"{figure_save}/{figure_name}_mean+sd.pdf")
plt.show()


# Part2: MSD analysis

### Step1 : Filter to 31 time points, process data for msd analysis

In [2]:
input_folder = "input_folder_name"
output_folder_31 = "output_folder_name" ## 31 time points correspond to the first 150 min
os.makedirs(output_folder_31, exist_ok=True)

def filter_to_first_31_rows(input_folder, output_folder):
    for file in os.listdir(input_folder):
        if file.endswith(".xlsx") and "spots_merged" in file:
            file_path = os.path.join(input_folder, file)
            print(f"🔹 Processing: {file}")
            df = pd.read_excel(file_path)
            filtered = (
                df.groupby("index")
                  .filter(lambda x: len(x) >= 31)
                  .groupby("index")
                  .head(31)
                  .reset_index(drop=True)
            )
            output_file = file.replace("_spots_merged.xlsx", "_spots_31points.xlsx")
            output_path = os.path.join(output_folder, output_file)
            filtered.to_excel(output_path, index=False)
            print(f"✅ Saved filtered file: {output_file}")

filter_to_first_31_rows(input_folder, output_folder_31)


### Step2 : MSD computation & α fitting 

In [ ]:

time_interval = 5

# Power-law model for alpha fitting #
def power_law(t, A, alpha):
    return A * t ** alpha

# Compute 1D MSD using PCA per droplet #
def compute_1d_msd_pca(df, time_col='time', pos_cols=['X', 'Y'], index_col='index', min_len=31):
    all_results = []
    for track_id, group in df.groupby(index_col):
        if len(group) < min_len:
            continue
        group = group.sort_values(by=time_col).reset_index(drop=True)
        pos = group[pos_cols].values
        pca = PCA(n_components=1)
        proj = pca.fit_transform(pos).flatten()
        n = len(proj)
        for tau in range(1, n):
            displacements = proj[tau:] - proj[:-tau]
            msd = np.mean(displacements**2)
            all_results.append({"index": track_id, "lag": tau * time_interval, "MSD_1D": msd})
    return pd.DataFrame(all_results)

# Fit alpha per 1D MSD curve 
def estimate_alpha_from_1d_msd(msd_df_1d):
    alpha_list = []
    for idx, group in msd_df_1d.groupby("index"):
        try:
            lags = group["lag"].values
            msd = group["MSD_1D"].values
            if np.max(msd) < 10:
                continue
            popt, _ = curve_fit(power_law, lags, msd, bounds=([0, 0], [np.inf, 3]))
            A, alpha = popt
            alpha_list.append({"index": idx, "A": A, "alpha_1D": alpha})
        except:
            continue
    return pd.DataFrame(alpha_list)

# Plot MSDs colored by alpha 
def plot_1d_msd_colored_by_alpha(msd_df_1d, alpha_df, label, save_dir):
    merged = pd.merge(msd_df_1d, alpha_df, on="index", how="inner")
    cmap = cm.coolwarm
    norm = Normalize(vmin=0, vmax=2)

    os.makedirs(save_dir, exist_ok=True)

    for scale in ['linear', 'loglog']:
        fig, ax = plt.subplots(figsize=(5, 3))
        plt.rcParams['font.family'] = 'Arial'

        for idx, group in merged.groupby("index"):
            alpha_val = group["alpha_1D"].iloc[0]
            ax.plot(group["lag"], group["MSD_1D"], color=cmap(norm(alpha_val)), alpha=0.5)

        mean_msd = merged.groupby("lag")["MSD_1D"].mean()
        ax.plot(mean_msd.index, mean_msd.values, color="black", linewidth=2, label="Mean MSD")

        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = fig.colorbar(sm, ax=ax)
        cbar.set_label("α (diffusion exponent)")

        plt.xlim (0,150)

        ax.set_title(f"1D MSD ({scale.capitalize()}): {label}",fontsize=12)
        ax.set_xlabel("Time lag (min)",fontsize=10)
        ax.set_ylabel("MSD (μm²)",fontsize=10)

        if scale == "loglog":
            ax.set_xscale("log")
            ax.set_yscale("log")
            ax.set_ylabel("MSD (μm², log)")

        ax.grid(True, which='both', ls='--')
        ax.legend()
        fig.tight_layout()

        fig.savefig(os.path.join(save_dir, f"{label}_1D_MSD_{scale}.pdf"))
        plt.close(fig)

def save_msd_and_alpha_to_excel(msd_df, alpha_df, output_path):
    with pd.ExcelWriter(output_path) as writer:
        msd_df.to_excel(writer, sheet_name="1D_MSD", index=False)
        alpha_df.to_excel(writer, sheet_name="alpha_summary", index=False)
    print(f"✅ Excel saved: {output_path}")
 
def run_pca_1d_analysis(input_df, label, save_dir):
    print(f"🔎 Analyzing: {label}")
    
    msd_1d_df = compute_1d_msd_pca(input_df)
    alpha_df = estimate_alpha_from_1d_msd(msd_1d_df)

    excel_path = os.path.join(save_dir, f"{label}_PCA_1D_MSD.xlsx")
    save_msd_and_alpha_to_excel(msd_1d_df, alpha_df, excel_path)

    plot_1d_msd_colored_by_alpha(msd_1d_df, alpha_df, label, save_dir)

    return msd_1d_df, alpha_df



### Step5: Run analysis on example dataset

In [ ]:
input_file = "///_spots_31points.xlsx" #define
save_folder = "folder_name" #define
condition_name = "condition_name" #define

df = pd.read_excel(input_file)

# Run full analysis
msd_df_1d, alpha_summary_1d = run_pca_1d_analysis(df, condition_name, save_folder)
